In [5]:
!pip install requests
!pip install rcsbsearchapi


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for tqdm from https://files.pythonhosted.org/packages/00/e5/f12a80907d0884e6dff9c16d0c0114d81b8cd07dc3ae54c5e962cc83037e/tqdm-4.66.1-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 10.8 MB/s eta 0:00:00
  Created wheel for rcsbsearchapi: filename=rcsbsearchapi-1.4.2-py2.py3-none-any.whl size=163536 sha256=32cc6be00810f4dc3b0b93958da7b6d2fffc89eac2a1cc1d6d4965b31c5349eb
  Stored in directory: /Users/ryan/Library/Caches/pip/wheels/d5/c7/25/f3e9f96e227b7524bf0777ada1f988378f3e2e1e066f0a9634
Successfully built rcsbsearchapi

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To u

In [14]:
import re
import requests

from rcsbsearchapi.search import SequenceQuery

### Phase 1: Solicit User for UniProt ID and Query/Save the Protein's Similar Sequences

In [28]:
# Define a function that takes a Uniprot ID as input and returns True if the ID exists in Uniprot and False if it doesn't.
def is_uniprot(pid):
    uniprot_api=f"https://www.uniprot.org/uniprot/{pid}.txt"
    response = requests.get(uniprot_api)
    if response.status_code == 200:
        return True
    else:
        pid = input(f"Error: {pid} doesn't exist in Uniprot. Enter a valid Uniprot ID:") # Get UniProt ID from user
        is_uniprot(pid)

# Define a function that takes a Uniprot ID as input and returns a list similar IDs from PubChem BioAssay.
def get_fasta(pid):
    uniprot_fasta_url = f"https://www.uniprot.org/uniprot/{pid}.fasta" # Need to update summary to similarity

    response = requests.get(uniprot_fasta_url)
    if response.status_code == 200:
        # Process the response if request was successful
        fasta = response.text
        return fasta
    else:
        # Handle errors
        print("Error:", response.text)

# Prompt the user for a UniProt ID
pid = input("Enter a Uniprot ID: ")
is_uniprot(pid)
fasta = get_fasta(pid)


None


In [71]:
# Function to query PDB for sequence similar proteins to the input protein. Returns a list of similar proteins.
def get_similarpid(fasta_sequence):
    search_request = {
        "query": {
            "type": "terminal",
            "service": "sequence",
            "parameters": {
            "evalue_cutoff": 1,
            "identity_cutoff": 0.9,
            "sequence_type": "protein",
            "value": fasta_sequence
            }
            },
        "request_options": {
                "scoring_strategy": "sequence"
            },
        "return_type": "polymer_entity"
    }

    # The json parameter in the requests.post automatically converts the Python dictionary to a JSON payload.
    response = requests.post("https://search.rcsb.org/rcsbsearch/v2/query", json=search_request)
    result = response.text
    return result

sequence = re.search(r'(?<=\n)[A-Z\n]+', fasta)
fasta_sequence = sequence.group(0).replace("\n", "")
results = get_similarpid(fasta_sequence)

print(results)

{
  "query_id" : "c1ecb5aa-ef6c-4b6b-9a03-aab98cdca196",
  "result_type" : "polymer_entity",
  "total_count" : 42,
  "result_set" : [ {
    "identifier" : "1YY4_1",
    "score" : 1.0
  }, {
    "identifier" : "1YYE_1",
    "score" : 1.0
  }, {
    "identifier" : "1QKM_1",
    "score" : 0.9410187667560321
  }, {
    "identifier" : "1L2J_1",
    "score" : 0.9249329758713136
  }, {
    "identifier" : "2I0G_1",
    "score" : 0.9195710455764075
  }, {
    "identifier" : "2JJ3_1",
    "score" : 0.9195710455764075
  }, {
    "identifier" : "2QTU_1",
    "score" : 0.9195710455764075
  }, {
    "identifier" : "2Z4B_1",
    "score" : 0.9195710455764075
  }, {
    "identifier" : "4ZI1_1",
    "score" : 0.8954423592493298
  }, {
    "identifier" : "5TOA_1",
    "score" : 0.8954423592493298
  } ]
}


### Phase 2: Query All Assays Related to all Identified Proteins from PubChem BioAssay

MDIKNSPSSLNSPSSYNCSQSILPLEHGSIYIPSSYVDSHHEYPAMTFYSPAVMNYSIPS
NVTNLEGGPGRQTTSPNVLWPTPGHLSPLVVHRQLSHLYAEPQKSPWCEARSLEHTLPVN
RETLKRKVSGNRCASPVTGPGSKRDAHFCAVCSDYASGYHYGVWSCEGCKAFFKRSIQGH
NDYICPATNQCTIDKNRRKSCQACRLRKCYEVGMVKCGSRRERCGYRLVRRQRSADEQLH
CAGKAKRSGGHAPRVRELLLDALSPEQLVLTLLEAEPPHVLISRPSAPFTEASMMMSLTK
LADKELVHMISWAKKIPGFVELSLFDQVRLLESCWMEVLMMGLMWRSIDHPGKLIFAPDL
VLDRDEGKCVEGILEIFDMLLATTSRFRELKLQHKEYLCVKAMILLNSSMYPLVTATQDA
DSSRKLAHLLNAVTDALVWVIAKSGISSQQQSMRLANLLMLLSHVRHASNKGMEHLLNMK
CKNVVPVYDLLLEMLNAHVLRGCKSSITGSECSPAEDSKSKEGSQNPQSQ



### Phase 3: Find Substances That Were Tested From Assays and Store in Dictionary

### Phase 4: Call Protein Data Bank for Pairs of InChi and UniProt Match 

### Phase 5: Output Remaining Data as a Table